In [1]:
# Allow reload of objects
%load_ext autoreload
%autoreload
from pelenet.utils import Utils
from pelenet.experiments.reward import RewardExperiment

# Official modules
import numpy as np
import nxsdk.api.n2a as nx
import matplotlib
import matplotlib.pyplot as plt
from time import time

In [2]:
# Initilizes the experiment and utils
utils = Utils.instance()
exp = RewardExperiment()

In [3]:
# Build network
exp.build()

# Get spectral radius
#utils.getSpectralRadius(exp.net.initialWeights)

In [4]:
# Prepare some variables
#nxNet = exp.net.nxNet

nExCores = int(np.ceil(exp.net.p.reservoirExSize / exp.net.p.neuronsPerCore))
nInCores = int(np.ceil(exp.net.p.reservoirInSize / exp.net.p.neuronsPerCore))
nAllCores = nExCores + nInCores

# Create compartment prototypes
compProto = nx.CompartmentPrototype(compartmentVoltageDecay=exp.net.p.compartmentVoltageDecay,
                                    refractoryDelay=exp.net.p.refractoryDelay, logicalCoreId=1)#nAllCores+1)

compProtoLrn = nx.CompartmentPrototype(compartmentVoltageDecay=exp.net.p.compartmentVoltageDecay,
                                       refractoryDelay=exp.net.p.refractoryDelay, logicalCoreId=1,#nAllCores+1,
                                       enableSpikeBackprop=1, enableSpikeBackpropFromSelf=1)


In [5]:
"""
Difference neuron
"""

diffCompPlus = exp.net.nxNet.createCompartmentGroup(size=1, prototype=compProto)
#diffCompPlus = exp.net.nxNet.createCompartment(prototype=compProto)
diffCompMinus = exp.net.nxNet.createCompartmentGroup(size=1, prototype=compProto)
#diffCompMinus = exp.net.nxNet.createCompartment(prototype=compProto)

In [6]:
"""
Output neuron
"""

#outComp = exp.net.nxNet.createCompartmentGroup(size=1, prototype=compProtoLrn)
outComp = exp.net.nxNet.createCompartment(prototype=compProtoLrn)

In [7]:
"""
Input generator
"""

inputSg = exp.net.nxNet.createSpikeGenProcess(numPorts=1)

# Generate spikes
rnd = np.random.rand(exp.net.p.totalSteps)
thresh = (exp.net.p.totalSteps-np.arange(exp.net.p.totalSteps))/exp.net.p.totalSteps
spike = (rnd < thresh)

# Index of every second spike
idxSecSpike = np.where(spike)[0][::2]

# Add spikes to generator
inputSg.addSpikes(spikeInputPortNodeIds=0, spikeTimes=list(idxSecSpike))

In [8]:
exConnProto = nx.ConnectionPrototype(weight=255, numTagBits=8, signMode=nx.SYNAPSE_SIGN_MODE.EXCITATORY)
inConnProto = nx.ConnectionPrototype(weight=255, numTagBits=8, signMode=nx.SYNAPSE_SIGN_MODE.INHIBITORY)

"""
Connect input, output, difference
"""
# Connect input to difference neurons
inputSg.connect(diffCompPlus, connectionMask=np.array([1]), weight=np.array([255]), prototype=exConnProto)
#inputSg.connect(diffCompPlus, prototype=exConnProto)
inputSg.connect(diffCompMinus, connectionMask=np.array([1]), weight=-np.array([255]), prototype=inConnProto)
#inputSg.connect(diffCompMinus, prototype=inConnProto)

# Connect output to difference neurons
#outComp.connect(diffCompPlus, connectionMask=np.array([1]), weight=np.array([255]), prototype=exConnProto)
outComp.connect(diffCompPlus, prototype=exConnProto)
#outComp.connect(diffCompMinus, connectionMask=np.array([1]), weight=-np.array([255]), prototype=inConnProto)
outComp.connect(diffCompMinus, prototype=inConnProto)

In [9]:
"""
Connect reservoir to output and add reward
"""

# Create learning rule and connection prototype for learning
lr = exp.net.nxNet.createLearningRule(dt='x1*y0-x0*y1-2^-2*u0*t', dw='2*u0*r1*t',
                                   x1Impulse=40, x1TimeConstant=4,
                                   y1Impulse=40, y1TimeConstant=4,
                                   r1Impulse=20, r1TimeConstant=4,
                                   tEpoch=2)
connLrnProto = nx.ConnectionPrototype(enableLearning=1, learningRule=lr, numTagBits=8,
                                      signMode=nx.SYNAPSE_SIGN_MODE.MIXED)

# Create mask and weights
ma = np.ones((1, exp.net.p.reservoirExSize)).astype(int)
we = np.random.normal(1, 5, exp.net.p.reservoirExSize).astype(int)

# Connect reservoir to output neuron
for i in range(len(exp.net.exReservoirChunks)):
    fr, to = i*exp.net.p.neuronsPerCore, (i+1)*exp.net.p.neuronsPerCore
    exp.net.exReservoirChunks[i].connect(outComp, connectionMask=ma[:,fr:to], weight=we[fr:to], prototype=connLrnProto)

# Connect difference neurons to reward channel
#inputSg.connect(lr.reinforcementChannel)
#diffCompPlus.connect(lr.reinforcementChannel)
diffCompPlus.connect(lr.reinforcementChannel, prototype=connLrnProto)
#diffCompMinus.connect(lr.reinforcementChannel)

AttributeError: 'Compartment' object has no attribute 'addSrcConnectionGroup'

In [ ]:
diffCompPlus

In [ ]:
lr.reinforcementChannel

In [ ]:
#dir(lr.reinforcementChannel)

In [ ]:
"""
Probes
"""

exp.net.outSpikeProbe = outComp.probe([nx.ProbeParameter.SPIKE])[0]

In [ ]:
plt.figure(figsize=(16,5))
plt.eventplot(idxSecSpike)

In [ ]:
exp.run()

In [ ]:
plt.figure(figsize=(16,5))
exp.net.outSpikeProbe.plot()

In [ ]:
np.sum(exp.net.outSpikeProbe.data)/1000

In [ ]:
exp.net.plot.reservoirSpikeTrain()

In [ ]:
exp.net.plot.reservoirRates()